In [1]:
import pandas as pd
import ollama
import random
import time

In [2]:
# import ptt title
ptt_df = pd.read_csv('../Data/Ptt/ptt_food_post_list.csv')
ptt_title = ptt_df['title']
ptt_title

0           [食記] 台南 來一片。一周只賣三天的美味披蕯店
1                 [食記] 台北 辰壽司割烹~大閘蟹宴
2            [食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶
3                 [食記] 新北林口｜林口三井的點點心
4          [食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯
                    ...             
1002             [食記] 台北 中山區 欣葉台菜創始店
1003    [食記] 台北信義-寓所咖啡Utroo cafe-咖啡廳
1004            [食記] 屏東潮州-牛大福。屏東牛肉料理
1005              [食記] 個人評比 肉次方VS夯下去
1006              [食記] 桃園龍潭 糧園茶藝客家小館
Name: title, Length: 1007, dtype: object

In [3]:
# prompt template
prompt_system_templates = [
    "你是美食部落客",
    "你是美食評論家",
    "你是美食熱愛者"
]

prompt_user_templates = [
    "寫一篇「{title}」為標題的文章，風格：PTT發文風格，要有餐廳名稱、地址、電話、營業時間、價位、消費日期或造訪年月(2022/11以前)",
    "以PTT美食版發文風格，寫一篇「{title}」為標題的文章，要包含以下資訊：餐廳名稱、地址、電話、營業時間、價位",
    "發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份，內容為餐廳環境、服務、食物的食用感想和價格，內容500-900字，標題為{title}",
    "在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引言不要打招呼，盡量詳細介紹每道菜的價格跟食用感想，標題為{title}"
]

In [4]:
# define the result_df
result_df = pd.DataFrame(columns=['title', 'content', 'prompt_system', 'prompt_user'])

In [5]:
title_list = ptt_title.tolist()
title_list

['[食記] 台南 來一片。一周只賣三天的美味披蕯店',
 '[食記] 台北 辰壽司割烹~大閘蟹宴',
 '[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶',
 '[食記] 新北林口｜林口三井的點點心',
 '[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯',
 '[食記] 桃園。麵屋虎千代',
 '[食記] [新竹] 少年G鹽水雞 ～少了胖焗多了滷味',
 '[食記] 台北信義 永春站 無敵水餃&搶鍋麵',
 '[食記] 基隆遠東泡泡冰 基隆仁愛 香綿雞蛋牛奶冰',
 '[食記] 台北 小黑碳鐵板漢堡',
 '[食記] 東京御徒町 鴨to蔥 鴨跟蔥的拉麵',
 '[食記] 台北松山-瑪黑餐酒',
 '[食記] 台北松山 湘民小館 CP值高平價湘菜',
 '[食記] 台北 士林區│牛魔王牛排館 士林店',
 "[食記] 台北松山 Will's Teppanyaki 好久沒來",
 '[食記] 新竹學府路美食 168新加坡美食',
 '[食記] 台北士林金蓬萊遵古台菜餐廳 解鎖預定菜',
 '[食記] 東京 後繼有人的佛心町洋食 三幸',
 '[食記] 羅東 築間幸福鍋物',
 '[食記] 台南 葡吉麵包店的奶露麵包和羅宋麵包',
 '[食記] 高雄前鎮｜華喜爌肉飯＿瑞隆路人氣名店',
 '[食記] 新加坡 東亞餐室 薄脆咖椰吐司與半熟蛋',
 '[食記] 桃園八德區。佳佳新埔粄條/湯圓',
 '[食記] 台北 釜山宗家韓式鍋物專門店',
 '[食記] 黃家麥克麵線 基隆中正 魚漿包大腸的麵線',
 '[食記] 桃園 龜山 欣越園越南美食館',
 '[食記] 台北大安 小胖好吃鹽水雞30年 靈魂雞汁',
 'Fw: [食記] 高雄 京厚屋豬排專賣(夢時代店)',
 '[食記] 台北 忠孝復興 滿粥穗',
 '[食記] 新竹恐龍造型雞蛋糕 Dino2石頁家脆皮雞蛋',
 '[食記] 金益鯊魚羹 新北瑞芳 軟絲膘配鯊魚羹米粉',
 '[食記] 台北串燒。南京復興站-京丘・炭火燒鳥',
 '[食記] 台北 貳樓 微風南山店',
 '[食記] 新竹 青葉客家麻糬 ～建中郵局前擺攤',
 '[食記] 金澤米其林二星壽司:めくみ 全日本頂尖',
 '[食記] 宜蘭礁溪 空ㄟ農場',
 '[食記] 台北 BEPPIN PASTA (來自日本靜岡)',
 '[食記] 台中 北屯區

In [6]:
failed_indices = []

count = 0
for title in title_list:
    count += 1
    print(count)
    system_template = random.choice(prompt_system_templates)
    user_template = random.choice(prompt_user_templates).format(title=title)

    response = None
    retries = 3  # 设置重试次数
    for attempt in range(retries):
        try:
            response = ollama.chat(
                model='ycchen/breeze-7b-instruct-v1_0',
                messages=[
                    {"role": "system", "content": system_template},
                    {"role": "user", "content": user_template}
                ]
            )
            break  # 成功获取响应后跳出重试循环
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(2)  # 等待2秒后重试
            else:
                print("Max retries reached. Skipping this title.")
                failed_indices.append(count - 1)  # 记录失败的索引
                continue

    if response:
        generated_content = response['message']['content']
        # 创建一个新的DataFrame行
        new_row = pd.DataFrame({
            'title': [title], 
            'content': [generated_content], 
            'prompt_system': [system_template], 
            'prompt_user': [user_template]
        })

        # 使用pandas.concat来添加新行
        result_df = pd.concat([result_df, new_row], ignore_index=True)

# 输出失败的索引
print("Failed indices:", failed_indices)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Attempt 1 failed with error: unexpected server status: llm busy - no slots available
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
Attempt 1 failed with error: unexpected server status: llm busy - no slots available
223
224
225
226
227
228
229
230
231
232
233
234
23

In [8]:
result_df.to_csv('../Data/LocalLLM/breeze_generated_articles.csv', index=False)